# 🤖 Smartest AI Nutrition Assistant - Multimodal & Multi-Agent System
This notebook supports:
- 📥 Inputs via **text**, **voice** (speech-to-text), or **image** (OCR from food labels or meals)
- 🥗 Personalized meal planning using GPT-4
- 💡 Nutritional explanations
- 🔄 Smart food swaps
- 🔁 Adaptive learning via user feedback

In [ ]:
!pip install openai pandas pytesseract SpeechRecognition pydub
import openai
import pandas as pd
import pytesseract
from PIL import Image
import speech_recognition as sr
from typing import Dict, Any
import io

In [ ]:
# 🔐 Set your OpenAI API Key
openai.api_key = 'your-api-key-here'

In [ ]:
# 🗣️ Voice input to text
def voice_to_text(audio_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)
    return recognizer.recognize_google(audio)

In [ ]:
# 🖼️ Extract text from images (OCR)
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    return pytesseract.image_to_string(image)

In [ ]:
# 👤 Sample user profile
user_profile = {
    'age': 30,
    'gender': 'female',
    'goals': 'fat loss',
    'medical_conditions': ['PCOS'],
    'fitness_routine': 'moderate gym 3x/week',
    'allergies': ['gluten', 'peanuts'],
    'diet_type': 'vegetarian',
    'calorie_target': 1600,
    'preferences': ['Indian', 'Thai']
}

In [ ]:
# 🧠 Agent class using GPT-4
class Agent:
    def __init__(self, name: str, role: str):
        self.name = name
        self.role = role

    def query(self, input_text: str) -> str:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": self.role},
                {"role": "user", "content": input_text}
            ]
        )
        return response.choices[0].message['content']

In [ ]:
# 🧠 Multi-Agent Definitions
meal_planner = Agent("MealPlanner", "You create meal plans based on user's health goals, medical conditions, and preferences.")
explainer = Agent("Explainer", "You explain why foods in a plan are good for user's health goals.")
swap_advisor = Agent("SwapAdvisor", "You suggest food swaps that are healthier and allergy-safe.")
feedback_updater = Agent("FeedbackAgent", "You update plans based on continuous user feedback.")

In [ ]:
# 🧩 Orchestrator Function
def orchestrate_system(user_profile: Dict[str, Any], feedback: str = None) -> Dict[str, str]:
    prompt = f"""
    Create a 1-day vegetarian meal plan for a {user_profile['age']} year old {user_profile['gender']} with {user_profile['medical_conditions']}.
    Fitness routine: {user_profile['fitness_routine']}. Goal: {user_profile['goals']}.
    Daily calorie limit: {user_profile['calorie_target']}.
    Allergies: {', '.join(user_profile['allergies'])}. Preferences: {', '.join(user_profile['preferences'])}.
    Return breakfast, lunch, dinner, snacks with portion sizes and calories.
    """
    plan = meal_planner.query(prompt)
    explanation = explainer.query(f"Explain why these foods help with {user_profile['goals']}:\n{plan}")
    swaps = swap_advisor.query(f"Suggest healthier or allergy-free swaps for:\n{plan}")

    if feedback:
        plan = feedback_updater.query(f"User says: {feedback}. Adjust this plan:\n{plan}")

    return {
        "meal_plan": plan,
        "explanation": explanation,
        "swaps": swaps
    }

In [ ]:
# ▶️ Example Execution
response = orchestrate_system(user_profile)
print("📋 Meal Plan:\n", response['meal_plan'])
print("💡 Explanation:\n", response['explanation'])
print("🔄 Swaps:\n", response['swaps'])